In [1]:
import pandas as pd
import numpy as np

In [2]:
data = {
    'Feature1': [0.5, 2.3, 1.1, 0.6, 2.1, 3.3, 0.8, 2.7, 1.4, 0.9],
    'Feature2': [1.2, 0.7, 3.3, 1.8, 0.4, 2.2, 1.9, 0.6, 3.0, 1.4],
    'Feature3': [3.3, 1.9, 0.4, 2.2, 3.1, 1.7, 2.8, 1.2, 0.9, 2.5],
    'Label1': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
    'Label2': [0, 1, 0, 1, 1, 0, 1, 1, 0, 1],
    'Label3': [1, 0, 1, 1, 0, 1, 1, 0, 0, 1]
}

# Membuat DataFrame
df = pd.DataFrame(data)
df

,Feature1,Feature2,Feature3,Label1,Label2,Label3
0,0.5,1.2,3.3,1,0,1
1,2.3,0.7,1.9,0,1,0
2,1.1,3.3,0.4,1,0,1
3,0.6,1.8,2.2,0,1,1
4,2.1,0.4,3.1,1,1,0
5,3.3,2.2,1.7,0,0,1
6,0.8,1.9,2.8,1,1,1
7,2.7,0.6,1.2,0,1,0
8,1.4,3.0,0.9,1,0,0
9,0.9,1.4,2.5,0,1,1


In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(df)

df_normalized = pd.DataFrame(scaler.transform(df), columns=df.columns)
df_normalized

,Feature1,Feature2,Feature3,Label1,Label2,Label3
0,0.000000,0.275862,1.000000,1.0,0.0,1.0
1,0.642857,0.103448,0.517241,0.0,1.0,0.0
2,0.214286,1.000000,0.000000,1.0,0.0,1.0
3,0.035714,0.482759,0.620690,0.0,1.0,1.0
4,0.571429,0.000000,0.931034,1.0,1.0,0.0
5,1.000000,0.620690,0.448276,0.0,0.0,1.0
6,0.107143,0.517241,0.827586,1.0,1.0,1.0
7,0.785714,0.068966,0.275862,0.0,1.0,0.0
8,0.321429,0.896552,0.172414,1.0,0.0,0.0
9,0.142857,0.344828,0.724138,0.0,1.0,1.0


In [4]:
df_feature = df_normalized.iloc[:, :3]
df_label = df_normalized.iloc[:, 3:]

In [5]:
X = df_feature.to_numpy()
y = df_label.to_numpy()

In [6]:
weight_list = []
for i, label in enumerate(df_label.columns):
    values_1 = df_label[label][df_label[label] == 1]
    values_2 = df_label[label][df_label[label] == 2]
    dist = 1 - (len(values_1) / len(df_label))
    weight_list.append(dist)

weight_list

[0.5, 0.4, 0.4]

In [7]:
import torch
import numpy as np
import random

def set_seed(seed):
    # Set the seed for generating random numbers
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    
    # Set deterministic options for CUDNN backend
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Example of setting the seed
set_seed(42)


In [20]:
import torch
import torch.nn as nn
import pandas as pd

class DNNModelwithWeightedDropout(nn.Module):
    def __init__(self, num_features=3, num_classes=3, p=0.5):
        super(DNNModelwithWeightedDropout, self).__init__()
        self.fcn1 = nn.Sequential(
            nn.Linear(num_features, 4, bias=False),
            nn.ReLU(),
            nn.Linear(4, 3, bias=False),
            nn.Softmax(dim=1)
        )

        self.drate = p

        self.dropout1 = nn.Dropout(p)
        self.dropout2 = nn.Dropout(p)
        self.dropout3 = nn.Dropout(p)

        self.fcn2 = nn.Sequential(
            nn.Linear(3, num_classes, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Create a dictionary to hold data for saving to Excel
        data = {}

        # Input to the network
        data["Input to the network"] = x.detach().numpy()

        # Forward pass through fcn1
        x = self.fcn1(x)
        data["Output of fcn1"] = x.detach().numpy()

        
        # Weights and biases of each layer in fcn1
        data["Weights of fcn1 layer 1 (Linear)"] = self.fcn1[0].weight.detach().numpy()
        # data["Biases of fcn1 layer 1 (Linear)"] = self.fcn1[0].bias.detach().numpy()
        data["Weights of fcn1 layer 2 (Linear)"] = self.fcn1[2].weight.detach().numpy()
        # data["Biases of fcn1 layer 3 (Linear)"] = self.fcn1[2].bias.detach().numpy()

        weights = x.detach().numpy()
        data["Weights for weighted dropout"] = weights

        mask = [(torch.bernoulli(torch.full(x[:, i].shape, 1 - self.drate))).detach().numpy() for i in range(3)]
        scaled_mask = [(mask[i] / (1 - self.drate)) for i in range(3)]
        output_manual = [(x[:, i] * scaled_mask[i]).detach().numpy() for i in range(3)]

        data["Mask"] = mask
        data["Scaled Mask"] = scaled_mask
        data["Dropout output"] = output_manual


        # Apply weighted dropout
        drop1 = self.dropout1(x[:, 0])
        drop2 = self.dropout2(x[:, 1])
        drop3 = self.dropout3(x[:, 2])

        channel1 = drop1 * weights[:, 0]
        channel2 = drop2 * weights[:, 1]
        channel3 = drop3 * weights[:, 2]


        data["Channel 1"] = x[:, 0].detach().numpy()
        data["Weights 1"] = weights[:, 0]

        data["Output dropout (channel 1)"] = drop1.detach().numpy()
        data["Output dropout (channel 2)"] = drop2.detach().numpy()
        data["Output dropout (channel 3)"] = drop3.detach().numpy()

        data["Output after weighted dropout (channel 1)"] = channel1.detach().numpy()
        data["Output after weighted dropout (channel 2)"] = channel2.detach().numpy()
        data["Output after weighted dropout (channel 3)"] = channel3.detach().numpy()

        x = torch.stack([channel1, channel2, channel3], dim=1)
        data["Stacked output after dropout"] = x.detach().numpy()

        # Forward pass through fcn2
        x = self.fcn2(x)
        data["Output of fcn2"] = x.detach().numpy()
        
        # Weights and biases of each layer in fcn2
        data["Weights of fcn2 layer 1 (Linear)"] = self.fcn2[0].weight.detach().numpy()
        # data["Biases of fcn2 layer 1 (Linear)"] = self.fcn2[0].bias.detach().numpy()

        # Save data to Excel file
        df = pd.DataFrame({key: pd.Series(value.tolist() if hasattr(value, 'tolist') else value) for key, value in data.items()})
        df.to_excel("output.xlsx", index=False)

        return x


In [22]:
# Example usage:
model = DNNModelwithWeightedDropout()
input_tensor = torch.tensor(X, dtype=torch.float32)  # Example input tensor
with torch.no_grad():
    input_tensor.requires_grad_(False)
    output = model(input_tensor)